In [ ]:
import face_recognition
from sklearn import svm
import os
from PIL import Image, ImageDraw
from IPython.display import display, clear_output
import cv2
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import matplotlib.patches as patches

In [ ]:
encodings = []
names = []
process_this_frame = True
# Training directory
train_dir = os.listdir('./train_dir')

In [ ]:
for person in train_dir:
    pix = os.listdir("./train_dir/" + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("./train_dir/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

In [ ]:
print(names)

In [ ]:
# Create and train the SVC classifier
clf = svm.SVC(gamma='scale')
clf.fit(encodings,names)

video_capture = cv2.VideoCapture("./videos/harrypotter2.gif")
frame_count=0
# Load the test image with unknown faces into a numpy array
test_image = face_recognition.load_image_file('./examples/test_image.jpg')

show_img = Image.fromarray(test_image)
draw = ImageDraw.Draw(show_img)

while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    if process_this_frame:
    # Find all the faces in the test image using the default HOG-based model
        face_locations = face_recognition.face_locations(rgb_small_frame)
        no = len(face_locations)
        # print("Number of faces detected: ", no)

        # Predict all the faces in the test image using the trained classifier
        # print("Found:")
        for i in range(no):
            (top, right, bottom, left)=face_locations[i]
            test_image_enc = face_recognition.face_encodings(test_image)[i]
            name = clf.predict([test_image_enc])
            # print(*name)
            name=str(*name)
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [6]:
import youtube_dl

ModuleNotFoundError: No module named 'youtube_dl'